In [1]:
import sys
print(sys.executable)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re
import warnings

c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\credit-card-fraud-detection-model-FZHIqfLr-py3.13\Scripts\python.exe


# Data Exploration – Dataset Tables

In this section we will first try to explore our raw dataset files to understand 
its structure, columns, datatypes, and potential issues (missing values, duplicates, etc.), before we preform any merge.

<br><u> We have 2 tables: </u></br>
* **customers.csv**
* **credit_card_fraud.csv**

### <u> Data Exploration – Customers Dataset : </u>

In [2]:
transactions_df = pd.read_csv("C:/Users/User/Documents/GitHub/Credit_Card_Fraud_Detection_Model/data/credit_card_fraud.csv", index_col=0) #first column is index
customers_df = pd.read_csv("C:/Users/User/Documents/GitHub/Credit_Card_Fraud_Detection_Model/data/customers.csv", sep="|")

In [11]:
# Save once (binary format)
transactions_df.to_pickle("transactions.pkl")
customers_df.to_pickle("customers.pkl")
warnings.filterwarnings('ignore')


In [2]:
transactions_df = pd.read_pickle("transactions.pkl")
customers_df = pd.read_pickle("customers.pkl")

# Functions:

In [4]:
def compare_columns(df1, name1, df2, name2):
    """
    Compares columns between two DataFrames.
    Prints:
    1. Common columns and their data types in both datasets.
    2. Columns unique to each dataset.
    """
    set1, set2 = set(df1.columns), set(df2.columns)
    common = set1.intersection(set2)
    only_in_1 = set1 - set2
    only_in_2 = set2 - set1
    
    print("="*60)
    print("🔹 Common Columns with Data Types")
    if common:
        comparison = {
            "Column": [],
            f"{name1} dtype": [],
            f"{name2} dtype": []
        }
        for col in sorted(common):
            comparison["Column"].append(col)
            comparison[f"{name1} dtype"].append(df1[col].dtype)
            comparison[f"{name2} dtype"].append(df2[col].dtype)
        result = pd.DataFrame(comparison)
        print(result.to_string(index=False))
    else:
        print("No common columns found.")

    print("\n" + "="*60)
    print(f"🔹 Columns only in {name1}:")
    if only_in_1:
        print(sorted(list(only_in_1)))

    else:
        print("No unique columns found.")
    
    print("\n" + "="*60)
    print(f"🔹 Columns only in {name2}:")
    if only_in_2:
        print(sorted(list(only_in_2)))
    else:
        print("No unique columns found.")
    print("="*60)

    return common, only_in_1, only_in_2


In [5]:
def unique_values_report(df, name, max_rows=20):
    """
    Prints number of unique values for each column in a DataFrame.
    Shows top 'max_rows' columns sorted by uniqueness.
    """
    uniques = df.nunique().sort_values(ascending=False).reset_index()
    uniques.columns = ["Column", "Unique Values"]
    
    print(f"\n🔹 Unique Value Report for {name} (showing top {max_rows}):")
    print(uniques.head(max_rows).to_string(index=False))
    


In [6]:
def clean_punc(col):
    return (
        col.str.lower()  # lowercase
           .str.replace(r"[^a-z0-9\s]", "", regex=True)  # remove special chars
           .str.strip()  # remove leading/trailing spaces
    )



In [7]:
def column_summary(df):
    summary = pd.DataFrame({
        'count': df.shape[0],
        'nulls': df.isnull().sum(),
        'nulls%': df.isnull().mean() * 100,
        'cardinality': df.nunique(),
        'dtype': df.dtypes
    })
    return summary


In [8]:
def check_case_duplicates(df, column):
    """
    Checks if a column has values that differ only by letter case.
    Returns True if normalization (lower/upper) is recommended.
    """
    original_unique = set(df[column].dropna().unique())
    normalized_unique = set(df[column].dropna().str.lower().unique())
    
    # If sizes differ → case duplicates exist
    if len(original_unique) != len(normalized_unique):
        print(f"⚠️ Column '{column}' has case duplicates.")
        diff = len(original_unique) - len(normalized_unique)
        print(f"   → {diff} duplicate categories caused by case.")
        return True
    else:
        print(f"✅ Column '{column}' has no case duplicates.")
        return False


In [9]:
def check_column_specials(df, column, sample_size=50000, sample=5):
    """
    Checks one column for punctuation or special characters.
    Uses a sample for speed on large datasets.
    
    Parameters:
    - df: pandas DataFrame
    - column: column name (string)
    - sample_size: number of rows to sample
    - sample: number of example values to display if issues are found
    """
    # Take a sample for speed
    df_sample = df[column].dropna().sample(
        min(sample_size, df[column].dropna().shape[0]), random_state=42
    ).astype(str)
    
    pattern = re.compile(r"[^a-zA-Z0-9\s]")
    mask = df_sample.str.contains(pattern, na=False)
    count = mask.sum()
    
    if count > 0:
        print(f"⚠️ Column '{column}' has {count} rows with special characters (in sample).")
        print("   Examples:", df_sample[mask].unique()[:sample])
        return "Needs Cleaning"
    else:
        print(f"✅ Column '{column}' is clean (no special characters in sample).")
        return "Clean"


In [10]:
def add_remark(summary_table, col_name, remark):
    """
    Add a remark for a specific column in the summary_table.
    
    Parameters:
        summary_table (pd.DataFrame): The summary table with 'remark' column.
        col_name (str): Column name to mark.
        remark (str): Remark text to add.
    """
    summary_table.loc[summary_table.index == col_name, "remark"] = remark
    display(summary_table.sort_values("cardinality", ascending=False))


def status_check(summary_table):
    """
    Display all columns in summary_table that have no remark.
    Works for both empty strings and NaN values.
    
    Parameters:
        summary_table (pd.DataFrame): The summary table with 'remark' column.
    """
    unmarked = summary_table[summary_table["remark"].isna() | (summary_table["remark"] == "")]
    display(unmarked.sort_values("cardinality", ascending=False))


## Pre-Merge Checks

- Name match  
- Data type match  
- Value overlap  
- Uniqueness  
- Nulls  
- Duplicates  


In [11]:
# Table Shape & Info:
transactions_df.shape, customers_df.shape

((34636378, 26), (20000, 16))

#### <font color='Navy'> 1.Columns & Data types inspection

In [12]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34636378 entries, 0 to 34636377
Data columns (total 26 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ssn         object 
 1   cc_num      int64  
 2   first       object 
 3   last        object 
 4   gender      object 
 5   street      object 
 6   city        object 
 7   state       object 
 8   zip         int64  
 9   lat         float64
 10  long        float64
 11  city_pop    int64  
 12  job         object 
 13  dob         object 
 14  acct_num    int64  
 15  profile     object 
 16  trans_num   object 
 17  trans_date  object 
 18  trans_time  object 
 19  unix_time   int64  
 20  category    object 
 21  amt         float64
 22  is_fraud    int64  
 23  merchant    object 
 24  merch_lat   float64
 25  merch_long  float64
dtypes: float64(5), int64(6), object(15)
memory usage: 7.0+ GB


In [13]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ssn       20000 non-null  object 
 1   cc_num    20000 non-null  int64  
 2   first     20000 non-null  object 
 3   last      20000 non-null  object 
 4   gender    20000 non-null  object 
 5   street    20000 non-null  object 
 6   city      20000 non-null  object 
 7   state     20000 non-null  object 
 8   zip       20000 non-null  int64  
 9   lat       20000 non-null  float64
 10  long      20000 non-null  float64
 11  city_pop  20000 non-null  int64  
 12  job       20000 non-null  object 
 13  dob       20000 non-null  object 
 14  acct_num  20000 non-null  int64  
 15  profile   20000 non-null  object 
dtypes: float64(2), int64(4), object(10)
memory usage: 2.4+ MB


#### <font color='navy'>
- Column Comparison - Name and Data Types
- Common Columns
- Nulls count check in both data sets


In [14]:
# Column Comparison - Name and Data Types:
common_cols, only_in_transactions, only_in_customers = compare_columns(
    transactions_df, "transactions", customers_df, "customers"
)

#Common columns:
print("Common columns:", common_cols) 

# Nulls check on common columns  - do we need the merge ? is the key column complete ?
print("Null counts in transactions:")
print(transactions_df[list(common_cols)].isna().sum())

print("Null counts in customers:")
print(customers_df[list(common_cols)].isna().sum())



🔹 Common Columns with Data Types
  Column transactions dtype customers dtype
acct_num              int64           int64
  cc_num              int64           int64
    city             object          object
city_pop              int64           int64
     dob             object          object
   first             object          object
  gender             object          object
     job             object          object
    last             object          object
     lat            float64         float64
    long            float64         float64
 profile             object          object
     ssn             object          object
   state             object          object
  street             object          object
     zip              int64           int64

🔹 Columns only in transactions:
['amt', 'category', 'is_fraud', 'merch_lat', 'merch_long', 'merchant', 'trans_date', 'trans_num', 'trans_time', 'unix_time']

🔹 Columns only in customers:
No unique columns found.
Common

KeyboardInterrupt: 

#### <font color='Indigo'> Data type Correction - Date type

In [ ]:
transactions_df['trans_date'] = pd.to_datetime(transactions_df['trans_date'], errors='coerce')
transactions_df['trans_time'] = pd.to_datetime(transactions_df['trans_time'], format='%H:%M:%S', errors='coerce')
transactions_df['dob'] = pd.to_datetime(transactions_df['dob'], errors='coerce')
customers_df['dob'] = pd.to_datetime(customers_df['dob'], errors='coerce')


In [ ]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34636378 entries, 0 to 34636377
Data columns (total 26 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ssn         object        
 1   cc_num      int64         
 2   first       object        
 3   last        object        
 4   gender      object        
 5   street      object        
 6   city        object        
 7   state       object        
 8   zip         int64         
 9   lat         float64       
 10  long        float64       
 11  city_pop    int64         
 12  job         object        
 13  dob         datetime64[ns]
 14  acct_num    int64         
 15  profile     object        
 16  trans_num   object        
 17  trans_date  datetime64[ns]
 18  trans_time  datetime64[ns]
 19  unix_time   int64         
 20  category    object        
 21  amt         float64       
 22  is_fraud    int64         
 23  merchant    object        
 24  merch_lat   float64       
 25  merch_long  float64  

In [ ]:
trans_cards = set(transactions_df['cc_num'])
cust_cards = set(customers_df['cc_num'])

overlap = trans_cards.intersection(cust_cards)

print("Unique cc_num in transactions:", len(trans_cards))
print("Unique cc_num in customers   :", len(cust_cards))
print("Overlapping cc_num           :", len(overlap))
print("Overlap coverage in transactions: {:.2f}%".format(len(overlap) / len(trans_cards) * 100))
print("Overlap coverage in customers   : {:.2f}%".format(len(overlap) / len(cust_cards) * 100))


Unique cc_num in transactions: 19948
Unique cc_num in customers   : 20000
Overlapping cc_num           : 19948
Overlap coverage in transactions: 100.00%
Overlap coverage in customers   : 99.74%


We want to confirm that each cc_num in customers_df appears only once (one row per customer).
If there are duplicates, merging could cause row multiplication.

In [ ]:
# Count duplicate customer IDs
dup_customers = customers_df['cc_num'].duplicated().sum()

print("Total customers:", customers_df.shape[0])
print("Unique customer IDs:", customers_df['cc_num'].nunique())
print("Duplicate customer IDs:", dup_customers)

# If there are duplicates, show a few
if dup_customers > 0:
    display(customers_df[customers_df['cc_num'].duplicated(keep=False)].head())


Total customers: 20000
Unique customer IDs: 20000
Duplicate customer IDs: 0


we will use the cc_num as the key join factor. the following check is just to confirm that this key has no missing values in it for each table.

In [ ]:
print("Nulls in transactions cc_num:", transactions_df['cc_num'].isna().sum())
print("Nulls in customers cc_num   :", customers_df['cc_num'].isna().sum())


Nulls in transactions cc_num: 0
Nulls in customers cc_num   : 0


In [ ]:
print("Duplicate rows in transactions:", transactions_df.duplicated().sum())
print("Duplicate rows in customers   :", customers_df.duplicated().sum())


Duplicate rows in transactions: 0
Duplicate rows in customers   : 0


What was found:
- All columns in customers.csv already exist in transactions_df.
- common columns in both dfs have no nulls.
- Customers_df have no extra special columns that can contribute extra informaiton. 
- The only difference is that customers.csv has a few extra cc_num that don’t appear in transactions.
- Those extra customers are irrelevant for fraud detection, because fraud is defined at the transaction level.

### <u> Data Exploration – credit_card_fraud Dataset </u>


In [ ]:
df = transactions_df

Since there is no additional information given by the customer.csv, we will be working on the credict_cart_fraud.csv
1. Duplicate rows - completed in previous section. no duplicates.
2. Null inspection
3. Check object columns
4. drop if a column has only one unique value
5. Convert an objects into viable data 
6. Clean special characters and punctioation
7. convert gender
8. drop first and last name
9. group states
10. remove street?

In [ ]:
# Run on transactions_df
summary_table = column_summary(df)
display(summary_table.sort_values("nulls%", ascending=False))



,count,nulls,nulls%,cardinality,dtype
ssn,34636378,0,0.0,19948,object
cc_num,34636378,0,0.0,19948,int64
first,34636378,0,0.0,686,object
last,34636378,0,0.0,1000,object
gender,34636378,0,0.0,2,object
street,34636378,0,0.0,19948,object
city,34636378,0,0.0,5184,object
state,34636378,0,0.0,51,object
zip,34636378,0,0.0,10218,int64
lat,34636378,0,0.0,9860,float64


## Column Summary – Key Insights

- **No Nulls**: All columns have 0 null values → no imputation required at this stage.  
- **Dataset Size**: The dataset contains ~34.6M rows → operations can be slow, so sampling may be needed for EDA.  
- **High-Cardinality (ID-like) Columns**: `ssn`, `cc_num`, `acct_num`, `trans_num` have many unique values (up to 1 per row).  
  - These are identifiers and not useful for EDA/modeling beyond grouping.  
- **Categorical Columns (low/moderate cardinality)**:  
  - `gender` (2 values), `state` (51), `category` (14), `profile` (12) → well-suited for analysis.  
- **Medium/High-Cardinality Categoricals**:  
  - `city` (~5K), `job` (~600), `merchant` (~700) → useful but may need category reduction (grouping rare values).  
- **Datetime Columns**: `dob`, `trans_date`, `trans_time` are already converted.  
  - Useful for deriving `age`, `year`, `month`, `day_of_week`.  
- **Numeric Columns**:  
  - `amt` (transaction amount), `city_pop`, `lat/long`, `merch_lat/long`.  
  - Some have very high precision and may require binning or transformation.  

**➡️ Next Step:** Focus on reducing categories, deriving new time-based features, and preparing the dataset for EDA visualizations.  


# Columns Consider dropping:

In [ ]:
# Identify columns where every row has a unique value
columns_to_drop = df.columns[df.nunique() == df.shape[0]]


In [ ]:
print("Columns with all unique values (to consider dropping):", list(columns_to_drop))

Columns with all unique values (to consider dropping): ['trans_num']


### paired data checker:

In [ ]:
# checks whether the SSN (ssn) and credit card number (cc_num) always change together in a DataFrame (df).
def paired_data_checker(df, col1, col2):
    """
    Checks if two columns always change together in a DataFrame.
    Returns True if they always change together, False otherwise.
    """
    paired_changes = df[[col1, col2]].drop_duplicates()
    unique_col1 = df[col1].nunique()
    unique_col2 = df[col2].nunique()
    
    if len(paired_changes) == max(unique_col1, unique_col2):
        print(f"{col1} and {col2} always change together. consider dropping one of them.")
        return
    else:
        print(f"{col1} and {col2} do not always change together.")
        return

# ssn_ccnum_pairs = df[['ssn', 'cc_num']].drop_duplicates()
# if len(ssn_ccnum_pairs) == len(df[['ssn', 'cc_num']].drop_duplicates(subset=['ssn'])):
#     print("ssn and cc_num always change together. Check whether to delete the column.")
# else:
#     print("ssn and cc_num do not always change together.")
# #############  df = df.drop(columns=['ssn']) ############    

In [ ]:
paired_data_checker(df, 'ssn', 'cc_num')
paired_data_checker(df, 'acct_num', 'cc_num')

ssn and cc_num always change together. consider dropping one of them.
acct_num and cc_num always change together. consider dropping one of them.


## Handle objects:

In [ ]:
df = df.astype({col: 'string' for col in df.select_dtypes(include='object').columns})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34636378 entries, 0 to 34636377
Data columns (total 26 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ssn         string        
 1   cc_num      int64         
 2   first       string        
 3   last        string        
 4   gender      string        
 5   street      string        
 6   city        string        
 7   state       string        
 8   zip         int64         
 9   lat         float64       
 10  long        float64       
 11  city_pop    int64         
 12  job         string        
 13  dob         datetime64[ns]
 14  acct_num    int64         
 15  profile     string        
 16  trans_num   string        
 17  trans_date  datetime64[ns]
 18  trans_time  datetime64[ns]
 19  unix_time   int64         
 20  category    string        
 21  amt         float64       
 22  is_fraud    int64         
 23  merchant    string        
 24  merch_lat   float64       
 25  merch_long  float64  

In [ ]:
# df['ssn'].head(5) --> 750-09-7342
df['ssn'] = df['ssn'].str.replace('-', '', regex=False)
df['ssn'].head(5)

0    750097342
1    750097342
2    750097342
3    750097342
4    750097342
Name: ssn, dtype: string

###Geneder conversion

In [ ]:
# Convert 'F' to 1 and 'M' to 0
df['gender'] = df['gender'].replace({'F': '1', 'M': '0'}).astype(int)
df['gender'].value_counts()



gender
1    17929380
0    16706998
Name: count, dtype: int64

In [ ]:
df_city = df['city']
df_city.value_counts()

city
Houston        349782
Chicago        338207
Brooklyn       294986
Los Angeles    263621
Las Vegas      207079
                ...  
Church Hill         7
Ash                 7
Oakley              7
Piney Flats         7
Montesano           7
Name: count, Length: 5184, dtype: Int64

In [ ]:
df_city2 = df_city.str.upper()

In [ ]:
# check_case_duplicates(df, "city")
# check_case_duplicates(df, "state")
check_case_duplicates(df, "job")

'✅ Columns have no case duplicates.'


✅ Column 'job' has no case duplicates.


'✅ Columns have no case duplicates.'

# Handling strings columns:

1. Columns to remove - create variable that marks them ( actual removal can be later)
2. Extraction of additional data

In [ ]:
df_strs =df.select_dtypes(include='string')



In [ ]:
df_strs.head(3)

,ssn,first,last,street,city,state,job,profile,trans_num,category,merchant
0,750097342,Amanda,Morris,144 Bowers Route,Tracy,CA,Teaching laboratory technician,young_adults_female_urban.json,52226c099d060def61a7e0d3de214e4b,misc_net,"fraud_Moore, Dibbert and Koepp"
1,750097342,Amanda,Morris,144 Bowers Route,Tracy,CA,Teaching laboratory technician,young_adults_female_urban.json,6495d95c66d7d504d7352f2b8c105d3c,misc_net,fraud_Dach-Nader
2,750097342,Amanda,Morris,144 Bowers Route,Tracy,CA,Teaching laboratory technician,young_adults_female_urban.json,bc0bfcd9e0132b171e538192b0e27777,grocery_pos,fraud_Pacocha-O'Reilly


## 1) Cardinality Inspection:

2.1 Job Column:

In [ ]:
# Run on transactions_df
summary_table = column_summary(df_strs)
display(summary_table.sort_values("cardinality", ascending=False))


,count,nulls,nulls%,cardinality,dtype
trans_num,34636378,0,0.0,34636378,string[python]
street,34636378,0,0.0,19948,string[python]
ssn,34636378,0,0.0,19948,string[python]
city,34636378,0,0.0,5184,string[python]
last,34636378,0,0.0,1000,string[python]
merchant,34636378,0,0.0,693,string[python]
first,34636378,0,0.0,686,string[python]
job,34636378,0,0.0,639,string[python]
state,34636378,0,0.0,51,string[python]
category,34636378,0,0.0,14,string[python]


# 1) Columns to remove

In [ ]:
removal_columns = ["ssn", "first", "last", "street", "trans_num"]
#ssn - semsitive, no predictive power, unique per person
#first, last - no predictive power
#street - high cardinality, no predictive power
#trans_num - unique per transaction, no predictive power
summary_table.loc[summary_table.index.isin(removal_columns), "remark"] = "remove"
display(summary_table.sort_values("cardinality", ascending=False))

,count,nulls,nulls%,cardinality,dtype,remark
trans_num,34636378,0,0.0,34636378,string[python],remove
street,34636378,0,0.0,19948,string[python],remove
ssn,34636378,0,0.0,19948,string[python],remove
city,34636378,0,0.0,5184,string[python],NaN
last,34636378,0,0.0,1000,string[python],remove
merchant,34636378,0,0.0,693,string[python],NaN
first,34636378,0,0.0,686,string[python],remove
job,34636378,0,0.0,639,string[python],NaN
state,34636378,0,0.0,51,string[python],NaN
category,34636378,0,0.0,14,string[python],NaN


In [ ]:
status_check(summary_table)

,count,nulls,nulls%,cardinality,dtype,remark
city,34636378,0,0.0,5184,string[python],NaN
merchant,34636378,0,0.0,693,string[python],NaN
state,34636378,0,0.0,51,string[python],NaN
category,34636378,0,0.0,14,string[python],NaN
profile,34636378,0,0.0,12,string[python],NaN


In [ ]:
print(df_strs["job"].str.lower().str.strip().unique().tolist())

['teaching laboratory technician', 'scientist, clinical (histocompatibility and immunogenetics)', 'english as a second language teacher', 'exercise physiologist', 'metallurgist', 'adult nurse', 'designer, television/film set', 'contractor', 'haematologist', 'therapist, drama', 'surveyor, mining', 'acupuncturist', 'publishing copy', 'lecturer, higher education', 'broadcast engineer', 'dispensing optician', 'legal secretary', 'chief of staff', 'administrator', 'environmental health practitioner', 'surveyor, planning and development', 'community arts worker', 'chemist, analytical', 'geographical information systems officer', 'health and safety inspector', 'police officer', 'barrister', 'amenity horticulturist', 'engineer, manufacturing', 'writer', 'race relations officer', 'careers information officer', 'fish farm manager', 'engineer, electronics', 'civil service fast streamer', 'pension scheme manager', 'media buyer', 'armed forces training and education officer', 'scientist, research (m

In [ ]:
unique_vals = df_strs["job"].unique().tolist()
print(unique_vals)



['Teaching laboratory technician', 'Scientist, clinical (histocompatibility and immunogenetics)', 'English as a second language teacher', 'Exercise physiologist', 'Metallurgist', 'Adult nurse', 'Designer, television/film set', 'Contractor', 'Haematologist', 'Therapist, drama', 'Surveyor, mining', 'Acupuncturist', 'Publishing copy', 'Lecturer, higher education', 'Broadcast engineer', 'Dispensing optician', 'Legal secretary', 'Chief of Staff', 'Administrator', 'Environmental health practitioner', 'Surveyor, planning and development', 'Community arts worker', 'Chemist, analytical', 'Geographical information systems officer', 'Health and safety inspector', 'Police officer', 'Barrister', 'Amenity horticulturist', 'Engineer, manufacturing', 'Writer', 'Race relations officer', 'Careers information officer', 'Fish farm manager', 'Engineer, electronics', 'Civil Service fast streamer', 'Pension scheme manager', 'Media buyer', 'Armed forces training and education officer', 'Scientist, research (m

In [ ]:
def categorize_jobs(df, column, new_column="job_category"):
    """
    Categorizes jobs based on keywords in the job title.
    Adds a new column with high-level categories.
    """

    def assign_category(job):
        job = str(job).lower()  # ensure lowercase
        if any(word in job for word in ["nurse", "doctor", "surgeon", "dentist", "therapist", "pharmacist", "psychiatrist", "psychologist", "radiographer", "optician", "midwife", "paramedic", "biomedical", "oncologist", "immunologist", "pathologist", "health"]):
            return "healthcare"
        elif any(word in job for word in ["teacher", "lecturer", "professor", "educator", "education officer", "tutor", "school"]):
            return "education"
        elif any(word in job for word in ["engineer", "technician", "technologist", "architect", "surveyor", "scientist", "geologist", "chemist", "biologist", "researcher", "ecologist", "mathematician", "statistician", "physicist", "astronomer"]):
            return "science/engineering"
        elif any(word in job for word in ["lawyer", "barrister", "solicitor", "attorney", "legal", "judge", "magistrate"]):
            return "legal"
        elif any(word in job for word in ["accountant", "finance", "banker", "economist", "trader", "investment", "treasurer", "auditor", "actuary"]):
            return "finance"
        elif any(word in job for word in ["artist", "designer", "animator", "illustrator", "musician", "actor", "writer", "journalist", "editor", "photographer", "producer", "curator", "painter", "sculptor", "filmmaker"]):
            return "arts/media"
        elif any(word in job for word in ["manager", "consultant", "officer", "administrator", "coordinator", "executive", "director", "chief", "ceo", "cfo", "cio", "cto", "cmo", "coo"]):
            return "management/business"
        elif any(word in job for word in ["armed forces", "navy", "army", "air force", "military", "police", "firefighter", "security"]):
            return "public safety/military"
        elif any(word in job for word in ["agricultural", "farm", "horticulturist", "fisheries", "forester", "conservation", "ecologist", "gardener", "landscaper", "tree surgeon"]):
            return "agriculture/environment"
        elif any(word in job for word in ["it", "software", "developer", "programmer", "data scientist", "web", "computer", "cyber", "network", "systems", "applications", "ai", "machine learning"]):
            return "technology"
        else:
            return "other"

    # apply categorization
    df[new_column] = df[column].apply(assign_category)
    return df


In [ ]:
df = categorize_jobs(df, "job")
df[["job", "job_category"]].head(20)


,job,job_category
0,Teaching laboratory technician,science/engineering
1,Teaching laboratory technician,science/engineering
2,Teaching laboratory technician,science/engineering
3,Teaching laboratory technician,science/engineering
4,Teaching laboratory technician,science/engineering
5,Teaching laboratory technician,science/engineering
6,Teaching laboratory technician,science/engineering
7,Teaching laboratory technician,science/engineering
8,Teaching laboratory technician,science/engineering
9,Teaching laboratory technician,science/engineering


In [ ]:
df_strs =df.select_dtypes(include='string')


In [ ]:
add_remark(summary_table, "job", "Reduced Cardinality to job_category")

,count,nulls,nulls%,cardinality,dtype,remark
trans_num,34636378,0,0.0,34636378,string[python],remove
street,34636378,0,0.0,19948,string[python],remove
ssn,34636378,0,0.0,19948,string[python],remove
city,34636378,0,0.0,5184,string[python],NaN
last,34636378,0,0.0,1000,string[python],remove
merchant,34636378,0,0.0,693,string[python],NaN
first,34636378,0,0.0,686,string[python],remove
job,34636378,0,0.0,639,string[python],Reduced Cardinality to job_category
state,34636378,0,0.0,51,string[python],NaN
category,34636378,0,0.0,14,string[python],NaN


In [ ]:
status_check(summary_table)



,count,nulls,nulls%,cardinality,dtype,remark
city,34636378,0,0.0,5184,string[python],NaN
merchant,34636378,0,0.0,693,string[python],NaN
state,34636378,0,0.0,51,string[python],NaN
category,34636378,0,0.0,14,string[python],NaN
profile,34636378,0,0.0,12,string[python],NaN


In [ ]:
merchant= df_strs['merchant']
merchant.value_counts()


merchant
fraud_Kilback LLC                        116978
fraud_Kuhn LLC                           100661
fraud_Boyer PLC                           96383
fraud_Dickinson Ltd                       94573
fraud_Cormier LLC                         93873
                                          ...  
fraud_Johnston-Casper                     21871
fraud_McLaughlin, Armstrong and Koepp     21869
fraud_Hagenes, Hermann and Stroman        21857
fraud_Kris-Kertzmann                      21808
fraud_Reichert-Weissnat                   21773
Name: count, Length: 693, dtype: Int64

In [ ]:
check_column_specials(df_strs, 'merchant', sample_size=10000, sample=10)

⚠️ Column 'merchant' has 10000 rows with special characters (in sample).
   Examples: ['fraud_Hirthe-Beier' 'fraud_Schmidt and Sons'
 'fraud_Bahringer, Bergnaum and Quitzon'
 'fraud_Heathcote, Yost and Kertzmann' 'fraud_Raynor, Reinger and Hagenes'
 'fraud_Swaniawski, Lowe and Robel' 'fraud_Emard Inc'
 'fraud_Powlowski-Weimann' 'fraud_Smith-Stokes' 'fraud_Reichel Inc']


'Needs Cleaning'

### Issues with 'merchant':
- High cardinality (many unique values)
- High granularity 
- Punctionation and special keys

In [ ]:
df['merchant'] = df['merchant'].str.replace('fraud_', '').str.replace(',', '')

In [ ]:
df['merchant'].value_counts()

merchant
Kilback LLC                       116978
Kuhn LLC                          100661
Boyer PLC                          96383
Dickinson Ltd                      94573
Cormier LLC                        93873
                                   ...  
Johnston-Casper                    21871
McLaughlin Armstrong and Koepp     21869
Hagenes Hermann and Stroman        21857
Kris-Kertzmann                     21808
Reichert-Weissnat                  21773
Name: count, Length: 693, dtype: Int64

In [3]:
df.to_pickle("df_prep_str.pkl")


NameError: name 'df' is not defined

In [2]:
df = pd.read_pickle("df_prep_str.pkl")

In [ ]:
status_check(summary_table)


,count,nulls,nulls%,cardinality,dtype,remark
city,34636378,0,0.0,5184,string[python],NaN
merchant,34636378,0,0.0,693,string[python],NaN
state,34636378,0,0.0,51,string[python],NaN
category,34636378,0,0.0,14,string[python],NaN
profile,34636378,0,0.0,12,string[python],NaN


In [ ]:
df['profile']

In [ ]:
category_stats = (
    df.groupby('merchant')
      .agg(total_transactions=('merchant', 'count'),
           total_fraud=('is_fraud', 'sum'))
      .reset_index().sort_values(by='total_fraud', ascending=False))

print(category_stats)

                          merchant  total_transactions  total_fraud
573                     Schumm PLC               93760         1007
316                    Kilback LLC              116978         1006
349                       Kuhn LLC              100661         1002
70                       Boyer PLC               96383          964
201                    Goyette Inc               50986          959
..                             ...                 ...          ...
324                    Kling-Grant               28944           46
299                 Kemmer-Reinger               29336           45
511                     Rempel PLC               29551           44
292                Johnston-Casper               21871           43
568  Schroeder Wolff and Hermiston               22000           43

[693 rows x 3 columns]


Profile Clean

In [4]:
print(df["profile"].head(10))   # shows first 20 rows


0    young_adults_female_urban.json
1    young_adults_female_urban.json
2    young_adults_female_urban.json
3    young_adults_female_urban.json
4    young_adults_female_urban.json
5    young_adults_female_urban.json
6    young_adults_female_urban.json
7    young_adults_female_urban.json
8    young_adults_female_urban.json
9    young_adults_female_urban.json
Name: profile, dtype: string


In [5]:
# Remove the .json ending
df['profile'] = df['profile'].str.replace('.json', '', regex=False)

In [6]:
print(df['profile'].head(10))   # shows first 20 rows

0    young_adults_female_urban
1    young_adults_female_urban
2    young_adults_female_urban
3    young_adults_female_urban
4    young_adults_female_urban
5    young_adults_female_urban
6    young_adults_female_urban
7    young_adults_female_urban
8    young_adults_female_urban
9    young_adults_female_urban
Name: profile, dtype: string


In [7]:
df['profile'].unique()

<StringArray>
['young_adults_female_urban',   'young_adults_male_rural',
    'adults_2550_male_urban',  'adults_2550_female_urban',
   'young_adults_male_urban',    'adults_2550_male_rural',
 'young_adults_female_rural',  'adults_2550_female_rural',
  'adults_50up_female_rural',    'adults_50up_male_urban',
    'adults_50up_male_rural',  'adults_50up_female_urban']
Length: 12, dtype: string

In [9]:
df["age_group"] = df["profile"].str.split("_").str[:2].str.join("_")


In [10]:
df["age_group"].value_counts()

age_group
adults_2550     19218103
adults_50up     11369708
young_adults     4048567
Name: count, dtype: int64

In [11]:
df["location_profile"] = df["profile"].str.split("_").str[-1]


In [1]:
df["location_profile"].value_counts()

NameError: name 'df' is not defined

In [ ]:
df.to_pickle("df_prep_str.pkl")

## 2) Extraction of additional data:

In [ ]:
category_stats = (
    df.groupby('merchant')
      .agg(total_transactions=('merchant', 'count'),
           total_fraud=('is_fraud', 'sum'))
      .reset_index().sort_values(by='total_fraud', ascending=False))

print(category_stats)

In [ ]:
check_column_specials(df, "city")
# check_column_specials(transactions_df, "job")
# check_column_specials(transactions_df, "merchant")




✅ Column 'city' is clean (no special characters in sample).


'Clean'

# Extraction of additional data:

# Data Drops:
- Street? high grnularity
- First and Last name